<a href="https://colab.research.google.com/github/LEANHDUC2005/Neural-Network---ANN/blob/main/B%E1%BA%A3n_sao_c%E1%BB%A7a_Fastspeech2_Hifigan_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# FastSpeech 2 + HiFi-GAN: Training from scratch (LJSpeech)
# Compatible with Google Colab Free Tier (T4 GPU)

# Step 1: Install dependencies
!pip install torch torchaudio numpy matplotlib scipy tensorboard
!pip install phonemizer unidecode librosa


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
!apt-get install -y espeak-ng

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 34 not upgraded.
Need to get 4,526 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpcaudio0 amd64 1.1-6build2 [8,956 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 espeak-ng-data amd64 1.50+dfsg-10ubuntu0.1 [3,956 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libespeak-ng1 amd64 1.50+dfsg-10ubuntu0.1 [207 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 espeak-ng amd64 1.50+dfsg-1

In [17]:
!pip install montreal-forced-aligner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 406.6/406.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.0/80.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 88.4 MB/s eta 0:00:00


In [18]:
!mfa model download acoustic english
!mfa model download dictionary english


Traceback (most recent call last):
  File "/usr/local/bin/mfa", line 5, in <module>
    from montreal_forced_aligner.command_line.mfa import mfa_cli
  File "/usr/local/lib/python3.11/dist-packages/montreal_forced_aligner/__init__.py", line 4, in <module>
    import montreal_forced_aligner.acoustic_modeling as acoustic_modeling
  File "/usr/local/lib/python3.11/dist-packages/montreal_forced_aligner/acoustic_modeling/__init__.py", line 7, in <module>
    from montreal_forced_aligner.acoustic_modeling.base import AcousticModelTrainingMixin  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/montreal_forced_aligner/acoustic_modeling/base.py", line 11, in <module>
    from _kalpy.gmm import AccumAmDiagGmm
ModuleNotFoundError: No module named '_kalpy'
Traceback (most recent call last):
  File "/usr/local/bin/mfa", line 5, in <module>
    from montreal_forced_aligner.command_line.mfa import mfa_cli
 

In [3]:
# Step 2: Clone HiFi-GAN for waveform decoder
!git clone https://github.com/jik876/hifi-gan.git
%cd hifi-gan
!pip install -r requirements.txt
%cd ..

Cloning into 'hifi-gan'...
remote: Enumerating objects: 48, done.
remote: Total 48 (delta 0), reused 0 (delta 0), pack-reused 48 (from 1)
Receiving objects: 100% (48/48), 620.94 KiB | 2.02 MiB/s, done.
Resolving deltas: 100% (20/20), done.
/content/hifi-gan
ERROR: Could not find a version that satisfies the requirement torch==1.4.0 (from versions: 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0)
ERROR: No matching distribution found for torch==1.4.0
/content


In [4]:
# Step 3: Download and extract LJSpeech dataset
!wget https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
!tar -xjf LJSpeech-1.1.tar.bz2

--2025-05-07 01:47:39--  https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
Resolving data.keithito.com (data.keithito.com)... 143.244.50.89, 2400:52e0:1a01::985:1
Connecting to data.keithito.com (data.keithito.com)|143.244.50.89|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2748572632 (2.6G) [text/plain]
Saving to: ‘LJSpeech-1.1.tar.bz2’

LJSpeech-1.1.tar.bz 100%[===================>]   2.56G   215MB/s    in 14s     

2025-05-07 01:47:53 (190 MB/s) - ‘LJSpeech-1.1.tar.bz2’ saved [2748572632/2748572632]



In [1]:
# Step 4: Define FastSpeech 2 components
import torch
import torch.nn as nn
import torch.nn.functional as F

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)].to(x.device)

class VariancePredictor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(input_dim, input_dim, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.5),
        )
        self.norm1 = nn.LayerNorm(input_dim)

        self.conv2 = nn.Sequential(
            nn.Conv1d(input_dim, input_dim, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.5),
        )
        self.norm2 = nn.LayerNorm(input_dim)

        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = x.transpose(1, 2)
        x = self.norm1(x)

        x = x.transpose(1, 2)
        x = self.conv2(x)
        x = x.transpose(1, 2)
        x = self.norm2(x)

        return self.linear(x).squeeze(-1)

def length_regulator(x, durations):
    output = []
    for i in range(x.size(0)):
        expanded = [x[i, j].repeat(int(max(durations[i][j], 1)), 1) for j in range(x.size(1))]
        output.append(torch.cat(expanded, dim=0))
    max_len = max([o.size(0) for o in output])
    output_padded = torch.stack([
        F.pad(o, (0, 0, 0, max_len - o.size(0))) for o in output
    ])
    return output_padded

class VarianceAdaptor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.duration_predictor = VariancePredictor(input_dim)
        self.pitch_predictor = VariancePredictor(input_dim)
        self.energy_predictor = VariancePredictor(input_dim)

    def forward(self, x):
        duration = self.duration_predictor(x)
        pitch = self.pitch_predictor(x)
        energy = self.energy_predictor(x)

        durations_rounded = torch.clamp(duration * 5, min=1).round().long()  # Boost duration prediction
        x = length_regulator(x, durations_rounded)

        return x, duration, pitch, energy

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(input_dim, hidden_dim, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.5),
        )
        self.norm = nn.LayerNorm(hidden_dim)

    def forward(self, x):
        x = self.conv(x.transpose(1, 2)).transpose(1, 2)
        x = self.norm(x)
        return x

class Decoder(nn.Module):
    def __init__(self, hidden_dim, mel_dim):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.5),
        )
        self.norm = nn.LayerNorm(hidden_dim)
        self.linear = nn.Linear(hidden_dim, mel_dim)

    def forward(self, x):
        x = self.conv(x.transpose(1, 2)).transpose(1, 2)
        x = self.norm(x)
        return self.linear(x)

class FastSpeech2(nn.Module):
    def __init__(self, vocab_size=300, d_model=256, mel_dim=80):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model)
        self.encoder = Encoder(d_model, d_model)
        self.variance_adaptor = VarianceAdaptor(d_model)
        self.decoder = Decoder(d_model, mel_dim)

    def forward(self, phoneme_ids):
        x = self.embedding(phoneme_ids)
        x = self.pos_enc(x)
        x = self.encoder(x)
        x, dur, pitch, energy = self.variance_adaptor(x)
        mel = self.decoder(x)
        return mel, dur, pitch, energy

class FastSpeech2(nn.Module):
    def __init__(self, vocab_size=300, d_model=256, mel_dim=80):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model)
        self.encoder = Encoder(d_model, d_model)
        self.variance_adaptor = VarianceAdaptor(d_model)
        self.decoder = Decoder(d_model, mel_dim)

    def forward(self, phoneme_ids):
        x = self.embedding(phoneme_ids)
        x = self.pos_enc(x)
        x = self.encoder(x)
        x, dur, pitch, energy = self.variance_adaptor(x)
        mel = self.decoder(x)
        return mel, dur, pitch, energy

NameError: name 'mel' is not defined

In [6]:
pip install --upgrade phonemizer


In [4]:
# Step 5: Preprocess LJSpeech to extract phonemes and mel-spectrograms
import os
import librosa
import numpy as np
from phonemizer import phonemize
from phonemizer.separator import Separator
from unidecode import unidecode
from tqdm import tqdm
import torchaudio
from phonemizer.backend import EspeakBackend
# Config
LJ_PATH = "LJSpeech-1.1"
SAMPLING_RATE = 22050
N_MELS = 80
HOP_LENGTH = 256

# Phoneme conversion
from phonemizer import phonemize
backend = EspeakBackend(language='en-us', preserve_punctuation=True)
def text_to_phonemes(text):
    return backend.phonemize([text], strip=True)[0]
    text = unidecode(text.strip())
    phones = phonemize(text, language='en-us', backend='espeak', separator=Separator(phone=' ', syllable=''))
    return phones

# Create mel-spectrogram
def wav_to_mel(wav_path):
    y, sr = librosa.load(wav_path, sr=SAMPLING_RATE)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=1024, hop_length=HOP_LENGTH, n_mels=N_MELS)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    return mel_db.T  # [T, 80]

# Prepare dataset (subset for quick demo)
metadata_path = os.path.join(LJ_PATH, "metadata.csv")
with open(metadata_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Pick a small subset for demo
data = []
for line in tqdm(lines[:100]):
    parts = line.strip().split("|")
    wav_path = os.path.join(LJ_PATH, "wavs", parts[0] + ".wav")
    text = parts[2]
    phones = text_to_phonemes(text)
    mel = wav_to_mel(wav_path)
    data.append((phones, mel))

#Save to disk (optional):
torch.save(data, "train_subset.pt")


100%|██████████| 100/100 [00:03<00:00, 32.40it/s]


In [ ]:
mv hifi_gan hifi-gan


In [8]:
# Step 6: Use pretrained HiFi-GAN (SpeechBrain) to synthesize .wav from mel
!pip install speechbrain --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 37.7 MB/s eta 0:00:00


In [5]:

import torchaudio
from speechbrain.pretrained import HIFIGAN

hifi_model = HIFIGAN.from_hparams(source="speechbrain/tts-hifigan-ljspeech", savedir="pretrained-hifigan")

# Function to synthesize waveform from mel
# Input shape: (n_mels=80, T)
def mel_to_wav(mel):
    with torch.no_grad():
        if mel.dim() == 2:
            mel = mel.unsqueeze(0)
        wav = hifi_model.decode_batch(mel).squeeze().cpu().numpy()
    return wav

# Step 7: Check compatibility
# Ensure FastSpeech 2 mel output is compatible with HiFi-GAN

def check_mel_specs(mel):
    assert mel.shape[1] > 0, "Mel-spectrogram has no time steps"
    assert mel.shape[2] == 80, f"Expected 80 mel channels, got {mel.shape[2]}"
    print("✔ Mel has 80 channels")

    sample_rate = 22050
    hop_length = 256
    print(f"✔ Target sample_rate = {sample_rate}, hop_length = {hop_length}")

# Example test
example_mel = torch.randn(1, 100, 80)  # [B, T, n_mels]
check_mel_specs(example_mel)

# Test synthesis
waveform = mel_to_wav(example_mel[0].transpose(0, 1))
print(f"Generated waveform shape: {waveform.shape}")

DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover
<ipython-input-5-3f7f0f0a73bb>:2: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import HIFIGAN
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Using symlink found at '/content/pretrained-hifigan/hyperparams.yaml'
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from Hugg

✔ Mel has 80 channels
✔ Target sample_rate = 22050, hop_length = 256
Generated waveform shape: (28160,)


In [6]:
import os
import random
import librosa
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

# --- Configuration ---
LJ_PATH = "LJSpeech-1.1"
SAMPLE_RATE = 22050
N_MELS = 80
HOP_LENGTH = 256

# --- Feature extraction ---
mel_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=SAMPLE_RATE,
    n_fft=1024,
    win_length=1024,
    hop_length=HOP_LENGTH,
    n_mels=N_MELS
)

def wav_to_mel(wav_path):
    wav, _ = librosa.load(wav_path, sr=SAMPLE_RATE)
    wav_tensor = torch.tensor(wav).unsqueeze(0)
    mel = mel_transform(wav_tensor).squeeze(0).transpose(0, 1)  # [T, n_mels]
    return mel

# --- Dummy phoneme tokenizer ---
def text_to_dummy_phonemes(text):
    return [ord(c) % 256 for c in text if c.isalnum()]

# --- Dataset and Dataloader ---
class LJSubset(Dataset):
    def __init__(self, max_samples=100):
        self.data = []
        with open(os.path.join(LJ_PATH, "metadata.csv"), "r", encoding="utf-8") as f:
            for line in f.readlines()[:max_samples]:
                parts = line.strip().split("|")
                wav_path = os.path.join(LJ_PATH, "wavs", parts[0] + ".wav")
                text = parts[2]
                phoneme_ids = text_to_dummy_phonemes(text)
                mel = wav_to_mel(wav_path)
                self.data.append((phoneme_ids, mel))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        phones, mel = self.data[idx]
        phones = torch.tensor(phones, dtype=torch.long)
        return phones, mel

def collate_fn(batch):
    phones_batch, mel_batch = zip(*batch)
    phones_padded = nn.utils.rnn.pad_sequence(phones_batch, batch_first=True, padding_value=0)
    mel_padded = nn.utils.rnn.pad_sequence(mel_batch, batch_first=True)
    return phones_padded, mel_padded

train_loader = DataLoader(LJSubset(), batch_size=4, shuffle=True, collate_fn=collate_fn)

# --- Train a few steps ---
model = FastSpeech2()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()




In [16]:
print(mel_pred.shape)  # e.g., torch.Size([batch_size, 131])
print(mel_target.shape)  # e.g., torch.Size([batch_size, 833])


torch.Size([4, 253, 80])
torch.Size([4, 838, 80])


In [ ]:
for batch in train_loader:
    phones, mels = batch
    mel_pred, dur, pitch, energy = model(phones)
    mel_pred = mel_pred[:, :mels.size(1), :]  # Align prediction with target length
    loss = loss_fn(mel_pred, mels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"Loss: {loss.item():.4f}")
    break

In [15]:
model.train()
for epoch in range(1):
    for i, (phones, mel_target) in enumerate(train_loader):
        mel_pred, _, _, _ = model(phones)
        loss = loss_fn(mel_pred, mel_target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 5 == 0:
            print(f"Epoch {epoch}, Step {i}, Loss = {loss.item():.4f}")
        if i == 20:
            break

# --- Test generation ---
model.eval()
test_input = torch.tensor([text_to_dummy_phonemes("hello world")], dtype=torch.long)
with torch.no_grad():
    mel_out, _, _, _ = model(test_input)
    print("Test mel shape:", mel_out.shape)
    check_mel_specs(mel_out)
    wav = mel_to_wav(mel_out[0].transpose(0, 1))

# --- Plot waveform ---
plt.figure(figsize=(12, 3))
plt.plot(wav)
plt.title("Generated Waveform")
plt.show()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([4, 838, 80])) that is different to the input size (torch.Size([4, 253, 80])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (253) must match the size of tensor b (838) at non-singleton dimension 1